In [8]:
from dolfin import *
import dolfin as df
from fenics import *

In [16]:
# Test for PETSc or Tpetra
if not has_linear_algebra_backend("PETSc") and not has_linear_algebra_backend("Tpetra"):
    info("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
    exit()

if not has_krylov_solver_preconditioner("amg"):
    info("Sorry, this demo is only available when DOLFIN is compiled with AMG "
         "preconditioner, Hypre or ML.")
    exit()

if has_krylov_solver_method("minres"):
    krylov_method = "minres"
elif has_krylov_solver_method("tfqmr"):
    krylov_method = "tfqmr"
else:
    info("Default linear algebra backend was not compiled with MINRES or TFQMR "
         "Krylov subspace method. Terminating.")
    exit()

In [17]:
# Define dimensions of the cubic domain
length_x = 0.01
length_y = 1
length_z = 1

# Define the number of elements in each dimension
num_elements_x = 1  # Number of elements in x-direction
num_elements_y = 100  # Number of elements in y-direction
num_elements_z = 100  # Number of elements in z-direction

# Create the mesh
mesh = BoxMesh(Point(0, 0, 0), Point(length_x, length_y, length_z), num_elements_x, num_elements_y, num_elements_z)

# Save the mesh in VTK format
with df.HDF5File(mesh.mpi_comm(), 'mesh/'+"mesh.h5", "w") as h5f:
    h5f.write(mesh, "mesh")

In [18]:
V = VectorFunctionSpace(mesh, 'P', 1)

In [19]:
u = Expression(('0', '0', '-1'), degree=1)

In [20]:
u = interpolate(u, V)

In [21]:
with HDF5File(mesh.mpi_comm(), "velocity/v_hdffive.h5", "w") as h5f:
    h5f.write(u, "u")

In [22]:
# Create XDMF files for visualization output
xdmffile_u = XDMFFile('velocity/v_show.xdmf')
xdmffile_u.parameters["flush_output"] = True
# Save solution to file (XDMF/HDF5)
xdmffile_u.write(u)